In [1]:
from datasets import Dataset, Audio
import os
import re
import json
import soundfile as sf
import numpy as np
import librosa

In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [3]:
SONGS_PATH = "new-songs"
LABELS_PATH = "new-labels"

In [4]:
audio_files = os.listdir(SONGS_PATH)

file_ids = [f.split(".")[0] for f in audio_files]
file_ids = list(filter(lambda x: len(x) > 0, file_ids))

In [5]:
audio_dataset = Dataset.from_dict({"audio": [os.path.join("new-songs", filename) for filename in audio_files]})
audio_dataset = audio_dataset.cast_column("audio", Audio(mono=True))

In [6]:
audio_dataset[0]

{'audio': {'path': 'new-songs/3130303230345f3135-0.wav',
  'array': array([0.14880371, 0.1972351 , 0.2727661 , ..., 0.1324768 , 0.12802124,
         0.12438965], dtype=float32),
  'sampling_rate': 44100}}

In [7]:
TARGET_SAMPLING_RATE = 16000
def attach_lyrics_and_resample(batch):
    batch["text"] = []
    new_audio = []
    
    for elem in batch["audio"]:
        # Bypass "input length too short" error
        if len(elem["array"]) == 0:
            continue
        
        # Read lyrics file
        audio_path = elem["path"]
        _, audio_filename = audio_path.split("/")
        filename, _ = audio_filename.split(".")
        
        # Attach lyrics
        label_path = os.path.join(LABELS_PATH, filename + ".json")
        with open(label_path) as f:
            label_data = f.read()
        label_data = json.loads(label_data)
        sent_text = " ".join([tok["d"] for tok in label_data["l"]])
        
        if len(re.sub(" ", "", sent_text)) == 0:
            continue
        
        batch["text"].append(sent_text)
        
        # Resample
        elem["array"] = librosa.resample(
            y=elem["array"], 
            orig_sr=elem["sampling_rate"], 
            target_sr=TARGET_SAMPLING_RATE)
        
        elem["sampling_rate"] = TARGET_SAMPLING_RATE
        elem["path"] = audio_path
        new_audio.append(elem)
        
    batch["audio"] = new_audio
    return batch

In [8]:
dataset_lr = audio_dataset.map(attach_lyrics_and_resample, batched=True, batch_size=100, num_proc=8)

Parameter 'indices'=range(0, 1154) of the transform datasets.arrow_dataset.Dataset.select couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


#0:   0%|          | 0/12 [00:00<?, ?ba/s]

#1:   0%|          | 0/12 [00:00<?, ?ba/s]

#2:   0%|          | 0/12 [00:00<?, ?ba/s]

#3:   0%|          | 0/12 [00:00<?, ?ba/s]

#4:   0%|          | 0/12 [00:00<?, ?ba/s]

#5:   0%|          | 0/12 [00:00<?, ?ba/s]

#6:   0%|          | 0/12 [00:00<?, ?ba/s]

#7:   0%|          | 0/12 [00:00<?, ?ba/s]

In [9]:
dataset_lr.push_to_hub("hungngocphat01/zalo-ai-train")

  0%|          | 0/5 [00:00<?, ?ba/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

Deleting unused files from dataset repository:   0%|          | 0/2 [00:00<?, ?it/s]

Updating downloaded metadata with the new split.
